## Создание таблицы с сигналами

In [13]:
import json
import csv
from pathlib import Path
import pandas as pd
import re


COLUMNS = ["KKS код", "Описание", "Ед. изм.", "Используемые сигналы", "Код"]


def iter_elements(elements):
    """elements может быть dict (как в описании) или list — обработаем оба варианта."""
    if isinstance(elements, dict):
        return elements.values()
    if isinstance(elements, list):
        return elements
    return []


def unique_preserve_order(items):
    seen = set()
    out = []
    for x in items:
        if not x:
            continue
        if x not in seen:
            seen.add(x)
            out.append(x)
    return out


def aggregate(folder: str):
    rows = []
    for path in sorted(Path(folder).glob("*.json")):
        try:
            data = json.loads(path.read_text(encoding="utf-8"))
        except Exception as e:
            # битые/не-JSON файлы пропускаем
            print(f"Skip {path.name}: {e}")
            continue

        project = data.get("project") or {}
        if project.get("type") != "parameter":
            continue

        signals = []
        for el in iter_elements(data.get("elements") or {}):
            if not isinstance(el, dict):
                continue
            if el.get("type") != "input-signal":
                continue
            props = el.get("props") or {}
            name = props.get("name")
            if isinstance(name, str):
                name = name.strip()
            elif name is not None:
                name = str(name)
            else:
                name = ""
            signals.append(name)

        signals_str = "; ".join(unique_preserve_order(signals))

        rows.append({
            "KKS код": project.get("code", ""),
            "Описание": project.get("description", ""),
            "Ед. изм.": project.get("dimension", ""),
            "Используемые сигналы": signals_str,
            "Код": data.get("code", ""),
        })

    return rows

def process_code_column(df):
    """
    Обрабатывает столбец 'Код':
    1. Заменяет AND->&&, OR->||, NOT->! (только как отдельные операторы)
    2. Заменяет § на _
    3. Для сигналов, начинающихся с цифры, добавляет 'P' в начало
    """
    
def process_code_column(df):
    """
    Обрабатывает столбец 'Код':
    1. Заменяет AND->&&, OR->||, NOT->! (только отдельные слова)
    2. Для сигналов, начинающихся с цифры, добавляет 'P' в начало
    3. Заменяет § на _
    """
    
    def process_row(row):
        code = row['Код']
        signals = row['Используемые сигналы']
        
        if pd.isna(code) or not isinstance(code, str) or code == "":
            return code
            
        # 1. Заменяем логические операторы как отдельные слова
        code = re.sub(r'\bAND\b', '&&', code)
        code = re.sub(r'\bOR\b', '||', code)
        code = re.sub(r'\bNOT\b', '!', code)
        
        # 2. Добавляем 'P' к сигналам, начинающимся с цифры
        if pd.notna(signals) and isinstance(signals, str) and signals != "":
            signal_list = [s.strip() for s in signals.split(';') if s.strip()]
            
            for signal in signal_list:
                if signal and signal[0].isdigit():
                    # ПРОСТО добавляем 'P' перед каждой найденной последовательностью
                    # Не анализируем контекст - просто заменяем все вхождения
                    code = code.replace(signal, 'P' + signal)
        
        # 3. Заменяем § на _
        code = code.replace('§', '_')
        
        return code
    
    df['Код'] = df.apply(process_row, axis=1)
    return df

def save_to_xlsx_with_formatting(df, out_path="result.xlsx"):
    # Сохраняем в XLSX
    df.to_excel(out_path, index=False, engine="openpyxl")
    
    from openpyxl import load_workbook
    from openpyxl.styles import Alignment

    wb = load_workbook(out_path)
    ws = wb.active

    # Включаем перенос текста для нужных столбцов
    wrap_cols = {
        "Описание",
        "Используемые сигналы",
        "Код",
    }

    # Найдём номера столбцов по заголовкам
    header = [cell.value for cell in next(ws.iter_rows(min_row=1, max_row=1))]
    col_idx = {name: i+1 for i, name in enumerate(header)}

    for col_name in wrap_cols:
        idx = col_idx.get(col_name)
        if not idx:
            continue
        for cell in ws.iter_cols(min_col=idx, max_col=idx, min_row=2):
            for c in cell:
                c.alignment = Alignment(wrap_text=True, vertical="top")

    # Немного подправим ширины столбцов (на вкус)
    widths = {
        "КKS код": 18,
        "Описание": 50,
        "Ед. изм.": 12,
        "Используемые сигналы": 30,
        "Код": 60,
    }
    for name, w in widths.items():
        idx = col_idx.get(name)
        if idx:
            ws.column_dimensions[ws.cell(row=1, column=idx).column_letter].width = w

    wb.save(out_path)



rows = aggregate('projects')
df = pd.DataFrame.from_records(rows).reindex(columns=COLUMNS)
df = process_code_column(df)
save_to_xlsx_with_formatting(df, 'Сигналы.xlsx')

## Запись правил в документ Word

In [9]:
import json
from pathlib import Path
import pandas as pd
from docx import Document
from docx.shared import Pt, Cm
from docx.enum.text import WD_ALIGN_PARAGRAPH

# iter_elements, unique_preserve_order, process_code_column должны быть уже определены выше

RULE_COLUMNS = ["КKS код", "Используемые сигналы", "Код", "Описание", "Методические указания"]
EXTRA_COLUMNS = ["params_list", "rule_code_raw"]

def aggregate_rules(folder: str):
    rows = []
    for path in sorted(Path(folder).glob("*.json")):
        try:
            data = json.loads(path.read_text(encoding="utf-8"))
        except Exception as e:
            print(f"Skip {path.name}: {e}")
            continue

        project = data.get("project") or {}
        if project.get("type") != "rule":
            continue

        signals = []
        params_list = []
        for el in iter_elements(data.get("elements") or {}):
            if not isinstance(el, dict):
                continue
            if el.get("type") not in ("input-signal", "table"):
                continue
            props = el.get("props") or {}
            name = props.get("name") or ""
            desc = props.get("description") or ""
            if isinstance(name, str):
                name = name.strip()
            else:
                name = str(name)
            params_list.append((name, desc))
            signals.append(name)
        signals_str = "; ".join(unique_preserve_order(signals))

        rows.append({
            "KKS код": project.get("code", ""),
            "Используемые сигналы": signals_str,
            "Код": data.get("code", ""),
            "Описание": project.get("possibleCause", ""),
            "Методические указания": project.get("guidelines", ""),
            "params_list": params_list,
            "rule_code_raw": project.get("code", ""),
        })
    return rows

def save_rules_to_docx(df, out_path="Правила.docx"):
    doc = Document()

    # Поля страницы
    section = doc.sections[0]
    section.top_margin = Cm(2)
    section.bottom_margin = Cm(2)
    section.left_margin = Cm(3)
    section.right_margin = Cm(1.5)

    def normalize_text(val):
        if val is None:
            return ""
        try:
            if isinstance(val, float) and pd.isna(val):
                return ""
        except Exception:
            pass
        s = str(val)
        s = s.replace("\r\n", "\n").replace("\r", "\n").replace("\\n", "\n")
        return s

    def add_paragraph_line(text="", bold=False):
        p = doc.add_paragraph()
        p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
        pf = p.paragraph_format
        pf.left_indent = Cm(0)
        pf.right_indent = Cm(0)
        pf.first_line_indent = Cm(1.25)
        pf.space_before = Pt(0)
        pf.space_after = Pt(0)
        pf.line_spacing = 1.5
        run = p.add_run(text)
        run.font.name = "Times New Roman"
        run.font.size = Pt(12)
        run.bold = bold
        return p

    def add_multiline(text="", bold=False):
        # Делим по \n и создаём отдельный абзац на каждую строку
        lines = normalize_text(text).split("\n")
        if not lines:
            add_paragraph_line("", bold=bold)
            return
        for line in lines:
            add_paragraph_line(line, bold=bold)

    for idx, row in df.iterrows():
        n = idx + 1
        rule_code = normalize_text(row.get("rule_code_raw"))
        code_text = normalize_text(row.get("Код"))
        descr = normalize_text(row.get("Описание"))
        guidelines = normalize_text(row.get("Методические указания"))
        params_list = row.get("params_list") or []

        # Правило N: (code)
        add_paragraph_line(f"Правило {n}: ({rule_code})", bold=True)

        # Краткое описание:
        add_paragraph_line("Краткое описание:", bold=True)
        add_multiline(descr, bold=False)

        # Причина:
        add_paragraph_line("Причина:", bold=True)
        add_paragraph_line("")  # пустая строка

        # Логика определения:
        add_paragraph_line("Логика определения:", bold=True)
        add_paragraph_line("")  # пустая строка

        # Формула:
        add_paragraph_line("Формула:", bold=True)
        add_multiline(code_text, bold=False)

        # Используемые параметры:
        add_paragraph_line("Используемые параметры:", bold=True)
        if params_list:
            for name, desc in params_list:
                name_t = normalize_text(name)
                desc_t = normalize_text(desc).replace("\n", "; ")
                line = f"- {name_t}"
                if desc_t:
                    line += f" — {desc_t}"
                add_paragraph_line(line, bold=False)
        else:
            add_paragraph_line("- (нет)", bold=False)

        # Время срабатывания:
        add_paragraph_line("Время срабатывания:", bold=True)
        add_paragraph_line("")  # пустая строка

        # Методические указания:
        add_paragraph_line("Методические указания:", bold=True)
        add_multiline(guidelines, bold=False)

        if idx != len(df) - 1:
            doc.add_page_break()

    doc.save(out_path)

# Агрегация, обработка кода и сохранение
rows_rule = aggregate_rules('projects')
df_rule = pd.DataFrame.from_records(rows_rule)
for col in RULE_COLUMNS + EXTRA_COLUMNS:
    if col not in df_rule.columns:
        df_rule[col] = ""
df_rule = df_rule[RULE_COLUMNS + EXTRA_COLUMNS]
df_rule = process_code_column(df_rule)
save_rules_to_docx(df_rule, "Правила.docx")
print(f"OK: {len(df_rule)} объектов -> Правила.docx")

OK: 10 объектов -> Правила.docx


## Запись сигналов в txt файл

In [17]:
import json
from pathlib import Path
import pandas as pd
from docx import Document
from docx.shared import Pt, Cm
from docx.enum.text import WD_ALIGN_PARAGRAPH

# iter_elements, unique_preserve_order, process_code_column должны быть определены выше
# process_code_column использует столбцы: "Используемые сигналы", "Код"

REQUIRED_COLS = ["KKS код", "Используемые сигналы", "Код", "Описание", "Методические указания"]

def aggregate_parameters(folder: str):
    rows = []
    for path in sorted(Path(folder).glob("*.json")):
        try:
            data = json.loads(path.read_text(encoding="utf-8"))
        except Exception as e:
            print(f"Skip {path.name}: {e}")
            continue

        project = data.get("project") or {}
        if project.get("type") != "parameter":
            continue

        signals = []
        for el in iter_elements(data.get("elements") or {}):
            if not isinstance(el, dict):
                continue
            if el.get("type") not in ("input-signal", "table"):
                continue
            props = el.get("props") or {}
            name = props.get("name")
            if isinstance(name, str):
                name = name.strip()
            elif name is not None:
                name = str(name)
            else:
                name = ""
            signals.append(name)

        signals_str = "; ".join(unique_preserve_order(signals))

        rows.append({
            "KKS код": project.get("code", ""),
            "Описание": project.get("description", ""),
            "Ед. изм.": project.get("dimension", ""),
            "Используемые сигналы": signals_str,
            "Код": data.get("code", ""),
            "Методические указания": project.get("guidelines", ""),
        })
    return rows

def normalize_text(val):
    if val is None:
        return ""
    try:
        if isinstance(val, float) and pd.isna(val):
            return ""
    except Exception:
        pass
    s = str(val)
    return s.replace("\r\n", "\n").replace("\r", "\n").replace("\\n", "\n")

def save_parameters_codes_docx(df, out_path="Параметры_Код.docx"):
    doc = Document()

    # Поля страницы (можно упростить/убрать при желании)
    section = doc.sections[0]
    section.top_margin = Cm(2)
    section.bottom_margin = Cm(2)
    section.left_margin = Cm(3)
    section.right_margin = Cm(1.5)

    def add_paragraph_line(text="", bold=False):
        p = doc.add_paragraph()
        p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
        pf = p.paragraph_format
        pf.left_indent = Cm(0)
        pf.right_indent = Cm(0)
        pf.first_line_indent = Cm(1.25)
        pf.space_before = Pt(0)
        pf.space_after = Pt(0)
        pf.line_spacing = 1.5
        run = p.add_run(text)
        run.font.name = "Times New Roman"
        run.font.size = Pt(12)
        run.bold = bold
        return p

    def add_multiline(text="", bold=False):
        lines = normalize_text(text).split("\n")
        # даже если пусто, добавим пустой абзац
        for line in lines if lines else [""]:
            add_paragraph_line(line, bold=bold)

    for _, row in df.iterrows():
        project_code = normalize_text(row.get("KKS код", ""))
        code_text = normalize_text(row.get("Код", ""))

        # project["code"]
        add_paragraph_line(project_code, bold=True)
        # обработанный "Код"
        add_multiline(code_text, bold=False)
        # разделитель
        add_paragraph_line("---------------------", bold=False)

    doc.save(out_path)
    print(f"DOCX сохранён: {out_path}, записано {len(df)} проектов")

# Агрегация
rows_param = aggregate_parameters('projects')  # при необходимости замените путь
df_param = pd.DataFrame.from_records(rows_param)

# Гарантируем наличие столбцов для process_code_column
for col in REQUIRED_COLS:
    if col not in df_param.columns:
        df_param[col] = ""

# Обработка кода (AND/OR/NOT, префикс P для сигналов с цифры, § -> _)
df_param = process_code_column(df_param)

# Сохраняем в DOCX
save_parameters_codes_docx(df_param, "Параметры_Код.docx")

DOCX сохранён: Параметры_Код.docx, записано 51 проектов


In [16]:
#!/usr/bin/env python3
"""
Миграция имён сигналов/таблиц: точки → подчёркивания
Обрабатывает все json файлы проектов с правильной структурой.
"""

import json
import re
from pathlib import Path
from typing import Dict, Any, List, Tuple, Set

# ============= НАСТРОЙКИ =============
PROJECTS_DIR = Path("proj_new")  # папка с проектами
DRY_RUN = False                   # True = тестовый режим (не записывать)
BACKUP = True                     # создавать .bak файлы перед изменением
VERBOSE = True                    # подробный вывод

# =====================================


def iter_elements(elements: Any) -> List[Dict[str, Any]]:
    """Поддержка dict и list для elements"""
    if isinstance(elements, dict):
        return list(elements.values())
    if isinstance(elements, list):
        return elements
    return []


def replace_identifiers_smart(text: str, mapping: Dict[str, str]) -> Tuple[str, int]:
    """
    Умная замена идентификаторов с учётом границ слов.
    Не трогает числа вида 12.5 и атрибуты вида np.mean
    Возвращает (новый текст, количество замен)
    """
    if not mapping or not isinstance(text, str):
        return text, 0
    
    replacement_count = 0
    
    # Сортируем по длине (от длинных к коротким) чтобы сначала заменить A.B.C, потом A.B
    sorted_items = sorted(mapping.items(), key=lambda x: len(x[0]), reverse=True)
    
    for old_name, new_name in sorted_items:
        if not old_name:
            continue
        
        # Паттерн: идентификатор не продолжение другого идентификатора
        # Слева не может быть: буква, цифра, _, .
        # Справа не может быть: буква, цифра, _, .
        pattern = rf'(?<![A-Za-z0-9_\.]){re.escape(old_name)}(?![A-Za-z0-9_\.])'
        
        def replacer(match):
            nonlocal replacement_count
            replacement_count += 1
            return new_name
        
        text = re.sub(pattern, replacer, text)
    
    return text, replacement_count


def collect_signal_names(elements: Any) -> Set[str]:
    """Собирает все имена input-signal и table"""
    names = set()
    for el in iter_elements(elements):
        if not isinstance(el, dict):
            continue
        if el.get("type") in ("input-signal", "table"):
            props = el.get("props") or {}
            name = props.get("name")
            if isinstance(name, str) and name:
                names.add(name)
    return names


def build_rename_mapping(names: Set[str]) -> Dict[str, str]:
    """Строит словарь переименований для имён с точками"""
    mapping = {}
    for name in names:
        if "." in name:
            new_name = name.replace(".", "_")
            mapping[name] = new_name
    return mapping


def process_project_file(filepath: Path) -> bool:
    """
    Обрабатывает один файл проекта.
    Возвращает True если были изменения.
    """
    try:
        # Читаем файл
        original_text = filepath.read_text(encoding="utf-8")
        data = json.loads(original_text)
    except Exception as e:
        print(f"❌ Ошибка чтения {filepath.name}: {e}")
        return False
    
    # Извлекаем структуру
    project_section = data.get("project") or {}
    elements = data.get("elements")
    root_code = data.get("code")  # Python-код на корневом уровне
    
    # === ШАГ 1: Собираем имена и строим mapping ===
    all_names = collect_signal_names(elements) if elements else set()
    rename_mapping = build_rename_mapping(all_names)
    
    if not rename_mapping and "." not in project_section.get("code", ""):
        if VERBOSE:
            print(f"⏭️  {filepath.name} — нет имён с точками")
        return False
    
    # Счётчики изменений
    stats = {
        "project_code": 0,
        "signal_names": 0,
        "formulas": 0,
        "python_code": 0
    }
    
    # === ШАГ 2: project.code (идентификатор проекта) ===
    # Это НЕ Python-код, а просто строка-идентификатор — можно менять в лоб
    if isinstance(project_section.get("code"), str) and "." in project_section["code"]:
        old_value = project_section["code"]
        project_section["code"] = old_value.replace(".", "_")
        stats["project_code"] = 1
        
        # Добавляем в mapping чтобы заменить в формулах тоже
        if old_value not in rename_mapping:
            rename_mapping[old_value] = project_section["code"]
    
    # === ШАГ 3: Переименовываем props.name у input-signal и table ===
    if elements:
        for el in iter_elements(elements):
            if not isinstance(el, dict):
                continue
            
            el_type = el.get("type")
            if el_type in ("input-signal", "table"):
                props = el.get("props") or {}
                name = props.get("name")
                if isinstance(name, str) and name in rename_mapping:
                    props["name"] = rename_mapping[name]
                    el["props"] = props
                    stats["signal_names"] += 1
    
    # === ШАГ 4: Обновляем props.expression у formula ===
    if elements and rename_mapping:
        for el in iter_elements(elements):
            if not isinstance(el, dict):
                continue
            
            if el.get("type") == "formula":
                props = el.get("props") or {}
                expr = props.get("expression")
                if isinstance(expr, str):
                    new_expr, count = replace_identifiers_smart(expr, rename_mapping)
                    if count > 0:
                        props["expression"] = new_expr
                        el["props"] = props
                        stats["formulas"] += count
    
    # === ШАГ 5: Обновляем корневой code (Python-код) ===
    if isinstance(root_code, str) and rename_mapping:
        new_code, count = replace_identifiers_smart(root_code, rename_mapping)
        if count > 0:
            data["code"] = new_code
            stats["python_code"] = count
    
    # === ШАГ 6: Обновляем данные в структуре ===
    data["project"] = project_section
    if elements is not None:
        data["elements"] = elements
    
    # === Проверяем были ли изменения ===
    total_changes = sum(stats.values())
    if total_changes == 0:
        if VERBOSE:
            print(f"⏭️  {filepath.name} — нет изменений")
        return False
    
    # === Формируем новое имя файла ===
    # Используем project.code + project.type
    proj_code = project_section.get("code", filepath.stem.split("_")[0])
    proj_type = project_section.get("type", "")
    
    if proj_type:
        new_filename = f"{proj_code}_{proj_type}.json"
    else:
        # Если type не задан, просто заменяем точки в текущем имени
        new_filename = filepath.stem.replace(".", "_") + ".json"
    
    new_filepath = filepath.parent / new_filename
    
    # === Вывод информации ===
    print(f"\n{'[DRY RUN] ' if DRY_RUN else ''}✅ {filepath.name}")
    if new_filename != filepath.name:
        print(f"   → {new_filename}")
    print(f"   Изменения:")
    print(f"     • project.code:     {stats['project_code']}")
    print(f"     • signal names:     {stats['signal_names']}")
    print(f"     • formulas:         {stats['formulas']}")
    print(f"     • python code:      {stats['python_code']}")
    if rename_mapping and VERBOSE:
        print(f"   Переименования:")
        for old, new in sorted(rename_mapping.items()):
            print(f"     • {old} → {new}")
    
    # === Сохранение ===
    if not DRY_RUN:
        # Создаём бэкап если нужно
        if BACKUP and filepath.exists():
            backup_path = filepath.with_suffix(".json.bak")
            backup_path.write_text(original_text, encoding="utf-8")
            if VERBOSE:
                print(f"   Бэкап: {backup_path.name}")
        
        # Сохраняем изменения
        new_json = json.dumps(data, ensure_ascii=False, indent=2)
        new_filepath.write_text(new_json, encoding="utf-8")
        
        # Удаляем старый файл если имя изменилось
        if new_filepath != filepath and filepath.exists():
            filepath.unlink()
            if VERBOSE:
                print(f"   Удалён старый: {filepath.name}")
    
    return True


def main():
    """Главная функция"""
    print("=" * 60)
    print("МИГРАЦИЯ ИМЁН: точки → подчёркивания")
    print("=" * 60)
    
    if DRY_RUN:
        print("⚠️  ТЕСТОВЫЙ РЕЖИМ (DRY_RUN) - файлы не будут изменены")
    
    print(f"📁 Папка проектов: {PROJECTS_DIR.resolve()}")
    print()
    
    # Проверяем существование папки
    if not PROJECTS_DIR.exists():
        print(f"❌ Папка не найдена: {PROJECTS_DIR.resolve()}")
        return 1
    
    # Находим все json файлы
    json_files = list(PROJECTS_DIR.glob("*.json"))
    
    # Исключаем бэкапы
    json_files = [f for f in json_files if not f.name.endswith(".bak")]
    
    if not json_files:
        print("⚠️  JSON файлы не найдены")
        return 0
    
    print(f"🔍 Найдено файлов: {len(json_files)}")
    print("-" * 60)
    
    # Обрабатываем каждый файл
    processed_count = 0
    error_count = 0
    
    for filepath in sorted(json_files):
        try:
            if process_project_file(filepath):
                processed_count += 1
        except Exception as e:
            error_count += 1
            print(f"❌ Ошибка обработки {filepath.name}: {e}")
            import traceback
            if VERBOSE:
                traceback.print_exc()
    
    # Итоги
    print("-" * 60)
    print("ИТОГИ:")
    print(f"  • Обработано файлов: {processed_count}")
    print(f"  • Пропущено файлов:  {len(json_files) - processed_count - error_count}")
    print(f"  • Ошибок:            {error_count}")
    
    if DRY_RUN:
        print("\n⚠️  Это был ТЕСТОВЫЙ запуск. Для реальных изменений установите DRY_RUN = False")
    else:
        print("\n✅ Миграция завершена!")
        if BACKUP:
            print("💾 Созданы бэкапы (.json.bak)")
    
    return 0


main()

МИГРАЦИЯ ИМЁН: точки → подчёркивания
📁 Папка проектов: /home/tishchenkova@lofi.pgt/Документы/Coding/GEH2/proj_new

🔍 Найдено файлов: 100
------------------------------------------------------------

✅ 10MAA50.LEAKAGE_RULE_rule.json
   → 10MAA50_LEAKAGE_RULE_rule.json
   Изменения:
     • project.code:     1
     • signal names:     4
     • formulas:         3
     • python code:      14
   Переименования:
     • 10MAA50.LEAKAGE_RULE → 10MAA50_LEAKAGE_RULE
     • 10MAA50FF.SEAL_LEAKAGE → 10MAA50FF_SEAL_LEAKAGE
     • 10MAA50FF.SEAL_LEAKAGE_CHAR → 10MAA50FF_SEAL_LEAKAGE_CHAR
     • 10MAA50FP001.P_IN → 10MAA50FP001_P_IN
     • L0LBA10FF001.FLOWRATE → L0LBA10FF001_FLOWRATE
   Бэкап: 10MAA50.LEAKAGE_RULE_rule.json.bak
   Удалён старый: 10MAA50.LEAKAGE_RULE_rule.json

✅ 10MAA50FF.SEAL_LEAKAGE_parameter.json
   → 10MAA50FF_SEAL_LEAKAGE_parameter.json
   Изменения:
     • project.code:     1
     • signal names:     1
     • formulas:         1
     • python code:      2
   Переименования:
  

0